### Interactive visual analytics and dashboard
Build a dashboard to analyze launch records interactively with Plotly, Dash. You can then build an interactive map to analyze the launch site proximity with Folium.

#### Interactive visual analytics with Folium lab
* Task 1: Mark all launch sites on a map
* Task 2: Mark the success/failed launches for each site on the map
* Task 3: Calculate the distances b/w a launch site to its proximities

In [1]:
import folium
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

df_spacex = pd.read_csv(URL)
df_spacex.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
df_spacex = df_spacex[['Launch Site', 'Lat', 'Long', 'class']]

# the first() method returns the first n rows, based on the specified value
df_sites = df_spacex.groupby(['Launch Site'], as_index = False).first()
df_sites

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Mark all launch sites on a map

In [4]:
df_sites = df_sites[['Launch Site', 'Lat', 'Long']]

# create a Folium map object with initial center location to be NASA
nasa_coord = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location = nasa_coord, zoom_start = 5)

In [5]:
# create a blue circle at NASA's location
# with a popup label showing its name
circle = folium.Circle(nasa_coord, radius = 2000, color = '#35400', fill = True).add_child(folium.Popup('NASA Johnson Space Center'))

# create an icon showing its name
marker = folium.map.Marker(
        nasa_coord,
        # create an icon as a text label
        icon = DivIcon(
            icon_size = (20, 20),
            icon_anchor = (0, 0),
            html = '<div style = "font-size: 12; color: #d35400;"> <b> %s </b> <div>' % 'NASA JSC',
            )
        )
site_map.add_child(circle)
site_map.add_child(marker)

In [6]:
# create and add circles and markers for each launch site on the map

for lat, lng, site in zip(df_spacex.Lat,
                          df_spacex.Long,
                          df_spacex['Launch Site']):
    circle = folium.Circle([lat, lng], radius = 2000,
                           color = 'crimson',
                           fill = False).add_to(site_map)
    marker = folium.Marker([lat, lng], popup = site).add_to(site_map)
    
site_map

Are all launch sites in proximity to the Equator line? 

Are all launch sites in very close proximity to coast?

In [7]:
df_spacex.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


Mark the success/failed launches for each site on the map

Marker clusters can simplify a map containing many markers having the same coordinate.

In [8]:
marker_color = []
for row in df_spacex['class']:
    if row == 1:
        marker_color.append('green')
    else:
        marker_color.append('red')
    
df_spacex['Marker_Color'] = marker_color
df_spacex.head()

,Launch Site,Lat,Long,class,Marker_Color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [9]:
# instantiate a feature group for the launch sites:
feature_group = folium.map.FeatureGroup()

coordinates = []
# loop through the locations, and add them to the feature group:
for index, rows in df_spacex.iterrows():
    my_list = [rows.Lat, rows.Long]
    coordinates.append(my_list)
    coordinate = coordinates[index]
    
    feature_group.add_child(folium.features.CircleMarker(
        coordinate, radius = 100, color = 'orange', fill = False))

# an alternative for loop
for lat, lng, site in zip(df_spacex.Lat,
                          df_spacex.Long,
                          df_spacex['Launch Site']):
    folium.Marker(location = [lat, lng], popup = site).add_to(site_map)

# add features to map
site_map.add_child(feature_group)

from folium import plugins 

# create the MarkerCluster object and add it to the map
marker_cluster = plugins.MarkerCluster().add_to(site_map)

for lat, lng, site, label in zip(df_spacex.Lat,
                                 df_spacex.Long,
                                 df_spacex['Launch Site'],
                                 df_spacex.Marker_Color):
    folium.Marker(location = [lat, lng], popup = site,
                  icon = folium.Icon(color = label,
                                     icon = 'tint')
                 ).add_to(marker_cluster)
    
site_map

Calculate the distances between a launch site to its proximities

Add MousePosition on the map to get the coordinate of a point. In this way, you can obtain the current coordinates on the go.

In [10]:
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
    position = 'topright',
    seperators = 'Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digis = 20,
    prefix = 'Latitude/Longitude: ',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat_site, lng_site, lat_point, lng_point):

    # approximate radius of earth in km
    R = 6373
    
    lat_site = radians(lat_site)
    lng_site = radians(lng_site)
    lat_point = radians(lat_point)
    lng_point = radians(lng_point)
    
    d_lat = lat_site - lat_point
    d_lng = lng_site - lng_point
    
    a = sin(d_lat/2)**2 + cos(lat_site)*cos(lat_point)*sin(d_lng/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))
    
    distance = R*c
    return distance

In [12]:
lat_point = 28.56208
lng_point = -80.56772

lat_site = df_spacex.loc[0, 'Lat']
lng_site = df_spacex.loc[0, 'Long']
print(lat_site, lng_site)

d_coastline = calculate_distance(lat_site, lng_site, lat_point, lng_point)
print('Distance R = ', d_coastline)

28.56230197 -80.57735648
Distance R =  0.9417404813661251


In [13]:
# create a marker of the closest coastline point on the map
# to the selected launch site
# display distance using the icon property

d_marker = folium.map.Marker([lat_point, lng_point],
                             icon = DivIcon(
                                 icon_size = (20,20),
                                 icon_anchor = (0,0),
                                 html = '<div style = "font-size: 12; color: #d35400;"> <b> %s </b> </div>' % "{:10.2f} km".format(d_coastline),
                             )
                            )
site_map.add_child(d_marker)

# create the line and display it
d_line = folium.PolyLine([[lat_site, lng_site],[lat_point, lng_point]],
                         weight = 1,
                         popup = 'Min. distance to coastline').add_to(site_map)
site_map

In [14]:
# Mark the minimum distance to a nearby city, railway, and highway

# list of latitudes and longitudes for these points
l_lat = [28.40854,  28.5721,   28.56214]
l_lng = [-80.60678, -80.58526, -80.57059]
element = ['city', 'railway', 'highway']

for i in range(0, len(l_lat)):
    d_element = calculate_distance(lat_site, lng_site, l_lat[i], l_lng[i])
    print(d_element)
    d_elem_marker = folium.map.Marker([l_lat[i], l_lng[i]],
                                      icon = DivIcon(
                                 icon_size = (20,20),
                                 icon_anchor = (0,0),
                                 html = '<div style = "font-size: 12; color: #d35400;"> <b> %s </b> </div>' % "{:10.2f} km".format(d_element),
                             )
                            ).add_to(site_map)
    d_elem_line = folium.PolyLine([[lat_site, lng_site], [l_lat[i], l_lng[i]]],
                                  weight = 1,
                                  popup = 'Minimum distance to ' + element[i]).add_to(site_map)
    
site_map                                  

17.34313807958749
1.3356073576955867
0.6612830968849855
